In [79]:
import os
from urllib.request import urlretrieve
import urllib.request as urllib
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.ndimage.interpolation import shift
import shutil

In [60]:
#grab data from NY State Open Data
url = 'https://data.ny.gov/api/views/ekwu-khcy/rows.csv?accessType=DOWNLOAD' #turnstile data

turnstile = pd.read_csv(url)
turnstile.head()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,23:00:00,REGULAR,5572864,1881239
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,19:00:00,REGULAR,5572521,1881206
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,15:00:00,REGULAR,5571587,1881113
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,11:00:00,REGULAR,5571313,1881031
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,08:10:05,REGULAR,5571173,1880736


In [26]:
#export raw data for records
turnstile.to_csv('turnstile2016_raw.csv')

#move raw data file to dat folder
shutil.move(os.getcwd()+'/turnstile2016_raw.csv',os.getcwd()+'/dat')

In [5]:
turnstile.shape

(9492670, 11)

In [11]:
#check to make sure all the days are there 
print("missing data for: {} days".format(365 - turnstile['Date'].nunique()))

missing data for: 29 days


In [16]:
turnstile['Time'].unique() #get unique times

array(['23:00:00', '19:00:00', '15:00:00', ..., '07:24:08', '11:36:08',
       '07:07:14'], dtype=object)

In [17]:
turnstile['Time'].nunique()

85074

In [18]:
turnstile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9492670 entries, 0 to 9492669
Data columns (total 11 columns):
C/A                                                           object
Unit                                                          object
SCP                                                           object
Station                                                       object
Line Name                                                     object
Division                                                      object
Date                                                          object
Time                                                          object
Description                                                   object
Entries                                                       int64
Exits                                                         int64
dtypes: int64(2), object(9)
memory usage: 796.7+ MB


In [64]:
#grab the hour from the time string 

turnstile['hour'] = turnstile['Time'].str[:2]

In [20]:
turnstile.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,hour
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,23:00:00,REGULAR,5572864,1881239,23
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,19:00:00,REGULAR,5572521,1881206,19
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,15:00:00,REGULAR,5571587,1881113,15
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,11:00:00,REGULAR,5571313,1881031,11
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,08:10:05,REGULAR,5571173,1880736,08


In [21]:
turnstile['hour'].unique()

array(['23', '19', '15', '11', '08', '07', '03', '10', '21', '17', '13',
       '09', '05', '01', '18', '12', '04', '20', '16', '00', '22', '06',
       '14', '02'], dtype=object)

#### so there are a two different cycles that the turnstiles might be on that seem legitimate as opposed to data flaws:
####  03 --> 07 --> 11 --> 15 --> 19 --> 23
#### 00 --> 04 --> 08 --> 12 --> 16 --> 20
#### we just need a time stamp for the day so take the latest time, hour == 23 or 00

In [65]:
turnstile_day = turnstile[(turnstile['hour'] == '23') | (turnstile['hour'] == '00')]
turnstile_day['hour'].unique()

array(['23', '00'], dtype=object)

In [36]:
turnstile_day.info() #check memory usage

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102216 entries, 0 to 9492664
Data columns (total 12 columns):
C/A                                                           1102216 non-null object
Unit                                                          1102216 non-null object
SCP                                                           1102216 non-null object
Station                                                       1102216 non-null object
Line Name                                                     1102216 non-null object
Division                                                      1102216 non-null object
Date                                                          1102216 non-null object
Time                                                          1102216 non-null object
Description                                                   1102216 non-null object
Entries                                                       1102216 non-null int64
Exits                       

In [66]:
turnstile_day.columns

Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date',
       'Time', 'Description', 'Entries',
       'Exits                                                     ', 'hour'],
      dtype='object')

In [67]:
#trim the white space from this column
turnstile_day.rename(columns={'Exits                                                     ':'Exits'},inplace=True)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [68]:
#aggregate to the station and day level
turnstile_agg = turnstile_day.groupby(['Station','Date'])[['Entries','Exits']].sum()

In [69]:
turnstile_agg.reset_index(inplace=True) #reset index after group by

In [70]:
turnstile_agg.columns #check columns

Index(['Station', 'Date', 'Entries', 'Exits'], dtype='object')

In [71]:
turnstile_agg.head()

,Station,Date,Entries,Exits
0,1 AV,01/02/2016,1407802820,1469374400
1,1 AV,01/03/2016,1407817257,1469389492
2,1 AV,01/04/2016,1407839057,1469414011
3,1 AV,01/05/2016,1407861698,1469438036
4,1 AV,01/06/2016,1407885485,1469462526


In [52]:
 #create a sample station data set to se
turnstile_agg[turnstile_agg['Station'] == '1 AV'].to_csv('station_sample.csv')

In [72]:
#convert Date column to datetime so we can order it and see which days we're missing 
turnstile_agg['Date'] = pd.to_datetime(turnstile_agg['Date'])

In [73]:
turnstile_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92897 entries, 0 to 92896
Data columns (total 4 columns):
Station    92897 non-null object
Date       92897 non-null datetime64[ns]
Entries    92897 non-null int64
Exits      92897 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 2.8+ MB


In [74]:
#sort by station and date so we can change the data from cumulative
turnstile_agg.sort_values(['Station','Date'], inplace=True) 

In [75]:
def find_diff_prev_row(df_series_col):
    """Function to find the difference between to adjacent rows"""
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    col_diff = abs(col_array - col_array_shifted)

    return col_diff

In [80]:
# calculate difference for entries and exits
turnstile_agg['entries_diff'] = find_diff_prev_row(turnstile_agg['Entries'])
turnstile_agg['exit_diff'] = find_diff_prev_row(turnstile_agg['Exits'])

In [81]:
turnstile_agg.head()

,Station,Date,Entries,Exits,entries_diff,exit_diff
0,1 AV,2016-01-02,1407802820,1469374400,9223372035446972988,9223372035385401408
1,1 AV,2016-01-03,1407817257,1469389492,14437,15092
2,1 AV,2016-01-04,1407839057,1469414011,21800,24519
3,1 AV,2016-01-05,1407861698,1469438036,22641,24025
4,1 AV,2016-01-06,1407885485,1469462526,23787,24490


In [84]:
turnstile_agg = turnstile_agg[turnstile_agg['entries_diff'] > 0] #filter only positive values

In [92]:
turnstile_agg.nlargest(9000,columns='entries_diff')

,Station,Date,Entries,Exits,entries_diff,exit_diff
0,1 AV,2016-01-02,1407802820,1469374400,9223372035446972988,9223372035385401408
1815,116 ST,2016-01-22,1709965437,2070687435,22918372442,28298873918
1814,116 ST,2016-01-21,24628337879,30369561353,21290505838,26285620663
59161,GRD CNTRL-42 ST,2016-02-27,12683283092,3133120644,8490366761,1567337700
19174,42 ST-PORT AUTH,2016-11-06,15062219323,12476575002,7531147377,6238326071
19175,42 ST-PORT AUTH,2016-11-07,7531171325,6238336315,7531047998,6238238687
18950,42 ST-PORT AUTH,2016-03-05,7521339438,6229915343,7521289988,6229868275
18943,42 ST-PORT AUTH,2016-02-27,15042122720,12459340940,7521085493,6229693706
22988,57 ST-7 AV,2016-03-05,7324622868,6355501575,7324602696,6355501709
22981,57 ST-7 AV,2016-02-27,14649056982,12710999640,7324537953,6355499962


In [97]:
turnstile_agg[turnstile_agg['Station'] == 'GRD CNTRL-42 ST'].median()
#TIMES SQ-42 ST
#GRD CNTRL-42 ST

Entries         2.310352e+09
Exits           2.429437e+08
entries_diff    7.333550e+04
exit_diff       6.547600e+04
dtype: float64

In [98]:
#filter out records that are greater than 3x grand centrals median volume
turnstile_agg = turnstile_agg[turnstile_agg['entries_diff'] < 292000] 

In [101]:
turnstile_agg.shape

(84666, 6)

In [102]:
turnstile_agg['Date'].nunique()

335

In [103]:
turnstile_agg['Station'].nunique()

372

In [108]:
print("we should have {} node date pairs".format(372*365))
print("we only have {} node date pairs".format(turnstile_agg.shape))
print("We're missing {} days entirely".format(365 - 335))
print("which represents {} node-date pairs".format(30*372))
print("we're missing: {} node-date pairs from available data (ie. we have the date for some but not all)".format((335 * 372) - 84666))
print("percentage missing {}".format(1 - 84666 / (335 * 372)))

we should have 135780 node date pairs
we only have (84666, 6) node date pairs
We're missing 30 days entirely
which represents 11160 node-date pairs
we're missing: 39954 node-date pairs from available data (ie. we have the date for some but not all)
percentage missing 0.32060664419836304


In [109]:
#find the data here: https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49
station_locs = gpd.read_file(os.getcwd()+'/dat/'+'subway_stations.shp')

In [110]:
station_locs.head()

,line,name,notes,objectid,url,geometry
0,4-6-6 Express,Astor Pl,"4 nights, 6-all times, 6 Express-weekdays AM s...",1.0,http://web.mta.info/nyct/service/,POINT (-73.99106999861966 40.73005400028978)
1,4-6-6 Express,Canal St,"4 nights, 6-all times, 6 Express-weekdays AM s...",2.0,http://web.mta.info/nyct/service/,POINT (-74.00019299927328 40.71880300107709)
2,1-2,50th St,"1-all times, 2-nights",3.0,http://web.mta.info/nyct/service/,POINT (-73.98384899986625 40.76172799961419)
3,2-3-4,Bergen St,"4-nights, 3-all other times, 2-all times",4.0,http://web.mta.info/nyct/service/,POINT (-73.97499915116808 40.68086213682956)
4,3-4,Pennsylvania Ave,"4-nights, 3-all other times",5.0,http://web.mta.info/nyct/service/,POINT (-73.89488591154061 40.66471445143568)


In [115]:
#export to build dictionary mapper 
station_list = pd.DataFrame(station_locs['name'].unique())
station_list.to_csv('station_map.csv') 

In [117]:
#export to build dictionary mapper
turnstile_list = pd.DataFrame(turnstile_agg['Station'].unique())
turnstile_list.to_csv('turnstile_map.csv')

In [121]:
#dict mapper so shapefile and volume data can be merged
dict_map = {
    
    '1 AV':'1st Ave',
'103 ST':'103rd St',
'103 ST-CORONA':'103rd St - Corona Plaza',
'104 ST':'104th St',
'110 ST':'110th St',
'111 ST':'111th St',
'116 ST':'116th St',
'116 ST-COLUMBIA':'116th St - Columbia University',
'121 ST':'121st St',
'125 ST':'125th St',
'135 ST':'135th St',
'137 ST CITY COL':'137th St - City College',
'138/GRAND CONC':'138th St - Grand Concourse',
'14 ST':'14th St',
'14 ST-UNION SQ':'Union Sq - 14th St',
'145 ST':'145th St',
'149/GRAND CONC':'149th St - Grand Concourse',
'14TH STREET':'14th St',
'15 ST-PROSPECT':'15th St - Prospect Park',
'155 ST':'155th St',
'157 ST':'157th St',
'161/YANKEE STAD':'161st St - Yankee Stadium',
'163 ST-AMSTERDM':'163rd St - Amsterdam Av',
'167 ST':'167th St',
'168 ST':'168th St',
'169 ST':'169th St',
'170 ST':'170th St',
'174 ST':'174th St',
'174-175 STS':'174th-175th Sts',
'175 ST':'175th St',
'176 ST':'176th St',
'18 AV':'18th Ave',
'18 ST':'18th St',
'181 ST':'181st St',
'182-183 STS':'182nd-183rd Sts',
'183 ST':'183rd St',
'190 ST':'190th St',
'191 ST':'191st St',
'2 AV':'Lower East Side - 2nd Ave',
'20 AV':'20th Ave',
'207 ST':'207th St',
'21 ST':'21st St',
'21 ST-QNSBRIDGE':'21st St - Queensbridge',
'215 ST':'215th St',
'219 ST':'219th St',
'225 ST':'225th St',
'23 ST':'23rd St',
'231 ST':'231st St',
'233 ST':'233rd St',
'238 ST':'238th St',
'25 AV':'25th Ave',
'25 ST':'25th St',
'28 ST':'28th St',
'3 AV':'3rd Ave',
'3 AV 138 ST':'3rd Ave - 138th St',
'3 AV-149 ST':'3rd Ave - 149th St',
'30 AV':'30th Ave',
'33 ST':'33rd St',
'33 ST-RAWSON ST':'NaN',
'34 ST-HERALD SQ':'NaN',
'34 ST-HUDSON YD':'34th St - Hudson Yards',
'34 ST-PENN STA':'34th St - Penn Station',
'36 AV':'36th Ave',
'36 ST':'36th St',
'39 AV':'39th Ave',
'4 AV-9 ST':'4th Av - 9th St',
'40 ST LOWERY ST':'40th St',
'42 ST-BRYANT PK':'42nd St - Bryant Pk',
'42 ST-PORT AUTH':'42nd St - Port Authority Bus Term',
'45 ST':'45th St',
'46 ST':'46th St',
'46 ST BLISS ST':'NaN',
'47-50 STS ROCK':'47th-50th Sts - Rockefeller Ctr',
'49 ST':'49th St',
'4AV-9 ST':'4th Av - 9th St',
'5 AVE':'5th Ave - 53rd St',
'50 ST':'50th St',
'51 ST':'51st St',
'52 ST':'52nd St',
'53 ST':'53rd St',
'55 ST':'55th St',
'57 ST':'57th St',
'57 ST-7 AV':'NaN',
'59 ST':'59th St',
'59 ST COLUMBUS':'59th St - Columbus Circle',
'6 AV':'6th Ave',
'61 ST WOODSIDE':'NaN',
'63 DR-REGO PARK':'63rd Dr - Rego Park',
'65 ST':'65th St',
'66 ST-LINCOLN':'66th St - Lincoln Ctr',
'67 AV':'67th Ave',
'68ST-HUNTER CO':'68th St - Hunter College',
'69 ST':'69th St',
'7 AV':'7th Ave',
'71 ST':'71st St',
'72 ST':'72nd St',
'74 ST-BROADWAY':'74th St - Broadway',
'75 AV':'75th Ave',
'75 ST-ELDERTS':'75th St - Eldert Ln',
'77 ST':'77th St',
'79 ST':'79th St',
'8 AV':'8th Ave',
'8 ST-NYU':'8th St - NYU',
'80 ST':'80th St',
'81 ST-MUSEUM':'81st St',
'82 ST-JACKSON H':'82nd St - Jackson Hts',
'85 ST-FOREST PK':'85th St - Forest Pky',
'86 ST':'86th St',
'88 ST':'88th St',
'9 AV':'9th Ave',
'90 ST-ELMHURST':'90th St - Elmhurst Av',
'96 ST':'96th St',
'9TH STREET':'NaN',
'ALABAMA AV':'Alabama Ave',
'ALLERTON AV':'Allerton Ave',
'AQUEDUCT N.COND':'Aqueduct - North Conduit Av',
'AQUEDUCT RACETR':'Aqueduct Racetrack',
'ASTOR PL':'Astor Pl',
'ASTORIA BLVD':'Astoria Blvd',
'ASTORIA DITMARS':'Astoria - Ditmars Blvd',
'ATL AV-BARCLAY':"Atlantic Av - Barclay's Center",
'ATLANTIC AV':'Atlantic Ave',
'AVENUE H':'Ave H',
'AVENUE I':'Ave I',
'AVENUE J':'Ave J',
'AVENUE M':'Ave M',
'AVENUE N':'Ave N',
'AVENUE P':'Ave P',
'AVENUE U':'Ave U',
'AVENUE X':'Ave X',
"B'WAY-LAFAYETTE":'Broadway - Lafayette St',
'BAY 50 ST':'Bay 50th St',
'BAY PKWY':'Bay Pky',
'BAY RIDGE AV':'Bay Ridge Ave',
'BAY RIDGE-95 ST':'Bay Ridge - 95th St',
'BAYCHESTER AV':'Baychester Ave',
'BEACH 105 ST':'Beach 105th St',
'BEACH 25 ST':'Beach 25th St',
'BEACH 36 ST':'Beach 36th St',
'BEACH 44 ST':'Beach 44th St',
'BEACH 60 ST':'Beach 60th St',
'BEACH 67 ST':'Beach 67th St',
'BEACH 90 ST':'Beach 90th St',
'BEACH 98 ST':'Beach 98th St',
'BEDFORD AV':'Bedford Ave',
'BEDFORD PK BLVD':'Bedford Park Blvd',
'BEDFORD-NOSTRAN':'Bedford - Nostrand Aves',
'BERGEN ST':'Bergen St',
'BEVERLEY ROAD':'Beverly Rd',
'BEVERLY RD':'Beverly Rd',
'BLEECKER ST':'Bleecker St',
'BOROUGH HALL':'Borough Hall',
'BOTANIC GARDEN':'Botanic Garden',
'BOWERY':'Bowery',
'BOWLING GREEN':'Bowling Green',
'BRIARWOOD':'Briarwood - Van Wyck Blvd',
'BRIGHTON BEACH':'Brighton Beach',
'BROAD CHANNEL':'Broad Channel',
'BROAD ST':'Broad St',
'BROADWAY':'Broadway',
'BROADWAY JCT':'Broadway Junction',
'BRONX PARK EAST':'Bronx Park East',
'BROOK AV':'Brook Ave',
'BROOKLYN BRIDGE':'Brooklyn Bridge - City Hall',
'BUHRE AV':'Buhre Ave',
'BURKE AV':'Burke Ave',
'BURNSIDE AV':'Burnside Ave',
'BUSHWICK AV':'Bushwick - Aberdeen',
'CANAL ST':'Canal St',
'CANARSIE-ROCKAW':'Canarsie - Rockaway Pkwy',
'CARROLL ST':'Carroll St',
'CASTLE HILL AV':'Castle Hill Ave',
'CATHEDRAL PKWY':'Cathedral Pkwy (110th St)',
'CENTRAL AV':'Central Ave',
'CENTRAL PK N110':'Central Park North (110th St)',
'CHAMBERS ST':'Chambers St',
'CHAUNCEY ST':'Chauncey St',
'CHRISTOPHER ST':'Christopher St - Sheridan Sq',
'CHURCH AV':'Church Ave',
'CITY / BUS':'NaN',
'CITY HALL':'City Hall',
'CLARK ST':'Clark St',
'CLASSON AV':'Classon Ave',
'CLEVELAND ST':'Cleveland St',
'CLINTON-WASH AV':'Clinton - Washington Aves',
'CONEY IS-STILLW':'Coney Island - Stillwell Av',
'CORTELYOU RD':'Cortelyou Rd',
'CORTLANDT ST':'Cortlandt St',
'COURT SQ':'Court Sq',
'COURT SQ-23 ST':'Court Sq - 23rd St',
'CRESCENT ST':'Crescent St',
'CROWN HTS-UTICA':'Crown Hts - Utica Ave',
'CYPRESS AV':'Cypress Ave',
'CYPRESS HILLS':'Cypress Hills',
'DEKALB AV':'DeKalb Ave',
'DELANCEY/ESSEX':'Delancey St - Essex St',
'DITMAS AV':'Ditmas Ave',
'DYCKMAN ST':'Dyckman St',
"E 143/ST MARY'S":"E 143rd St - St Mary's St",
'E 149 ST':'E 149th St',
'E 180 ST':'E 180th St',
'EAST 105 ST':'E 105th St',
'EAST BROADWAY':'East Broadway',
'EASTCHSTER/DYRE':'Eastchester - Dyre Ave',
'EASTN PKWY-MUSM':'Eastern Pkwy - Bklyn Museum',
'ELDER AV':'Elder Ave',
'ELMHURST AV':'Elmhurst Ave',
'EUCLID AV':'Euclid Ave',
'EXCHANGE PLACE':'NaN',
'FAR ROCKAWAY':'Far Rockaway - Mott Ave',
'FLATBUSH AV-B.C':'Brooklyn College - Flatbush Ave',
'FLUSHING AV':'Flushing Ave',
'FLUSHING-MAIN':'Flushing - Main St',
'FORDHAM RD':'Fordham Rd',
'FOREST AVE':'Forest Ave',
'FOREST HILLS 71':'Forest Hills - 71st Av',
'FRANKLIN AV':'Franklin Ave',
'FRANKLIN ST':'Franklin St',
'FREEMAN ST':'Freeman St',
'FRESH POND RD':'Fresh Pond Rd',
'FT HAMILTON PKY':'Ft Hamilton Pkwy',
'FULTON ST':'Fulton St',
'GATES AV':'Gates Ave',
'GRAHAM AV':'Graham Ave',
'GRAND ARMY PLAZ':'Grand Army Plaza',
'GRAND ST':'Grand St',
'GRAND-NEWTOWN':'Grand Ave - Newtown',
'GRANT AV':'Grant Ave',
'GRD CNTRL-42 ST':'Grand Central - 42nd St',
'GREENPOINT AV':'Greenpoint Ave',
'GROVE STREET':'NaN',
'GUN HILL RD':'Gun Hill Rd',
'HALSEY ST':'Halsey St',
'HARLEM 148 ST':'Harlem - 148 St',
'HARRISON':'NaN',
'HEWES ST':'Hewes St',
'HIGH ST':'High St',
'HOUSTON ST':'Houston St',
'HOWARD BCH JFK':'Howard Beach - JFK Airport',
'HOYT ST':'Hoyt St',
'HOYT-SCHER':'Hoyt - Schermerhorn Sts',
'HUNTERS PT AV':'Hunters Point Ave',
'HUNTS POINT AV':'Hunts Point Ave',
'INTERVALE AV':'Intervale Ave',
'INWOOD-207 ST':'Inwood - 207th St',
'JACKSON AV':'Jackson Ave',
'JAMAICA 179 ST':'Jamaica - 179th St',
'JAMAICA CENTER':'Jamaica Ctr - Parsons / Archer',
'JAMAICA VAN WK':'Jamaica - Van Wyck',
'JAY ST-METROTEC':'Jay St - MetroTech',
'JEFFERSON ST':'Jefferson St',
'JFK JAMAICA CT1':'NaN',
'JKSN HT-ROOSVLT':'Jackson Hts - Roosevelt Av',
'JOURNAL SQUARE':'NaN',
'JUNCTION BLVD':'Junction Blvd',
'JUNIUS ST':'Junius St',
'KINGS HWY':'Kings Hwy',
'KINGSBRIDGE RD':'Kingsbridge Rd',
'KINGSTON AV':'Kingston Ave',
'KINGSTON-THROOP':'Kingston - Throop Aves',
'KNICKERBOCKER':'Knickerbocker Ave',
'KOSCIUSZKO ST':'Kosciuszko St',
'LACKAWANNA':'NaN',
'LAFAYETTE AV':'Lafayette Ave',
'LEXINGTON AV/53':'Lexington Ave - 53rd St',
'LEXINGTON AV/63':'Lexington Ave - 63rd St',
'LIBERTY AV':'Liberty Ave',
'LIVONIA AV':'Livonia Ave',
'LONGWOOD AV':'Longwood Ave',
'LORIMER ST':'Lorimer St',
'MARBLE HILL-225':'Marble Hill - 225th St',
'MARCY AV':'Marcy Ave',
'METROPOLITAN AV':'Metropolitan Ave',
'METS-WILLETS PT':'Mets - Willets Point',
'MIDDLETOWN RD':'Middletown Rd',
'MONTROSE AV':'Montrose Ave',
'MORGAN AV':'Morgan Ave',
'MORISN AV/SNDVW':'Morrison Av - Soundview',
'MORRIS PARK':'Morris Park',
'MOSHOLU PKWY':'Mosholu Pkwy',
'MT EDEN AV':'Mt Eden Ave',
'MYRTLE AV':'Myrtle Ave',
'MYRTLE-WILLOUGH':'Myrtle-Willoughby Aves',
'MYRTLE-WYCKOFF':'Myrtle - Wyckoff Aves',
'NASSAU ST':'Nassau Ave',
'NECK RD':'Neck Rd',
'NEPTUNE AV':'Neptune Ave',
'NEREID AV':'Nereid Ave (238 St)',
'NEVINS ST':'Nevins St',
'NEW LOTS':'New Lots Ave',
'NEW LOTS AV':'New Lots Ave',
'NEW UTRECHT AV':'New Utrecht Ave',
'NEWARK BM BW':'NaN',
'NEWARK C':'NaN',
'NEWARK HM HE':'NaN',
'NEWARK HW BMEBE':'NaN',
'NEWKIRK AV':'Newkirk Ave',
'NEWKIRK PLAZA':'NaN',
'NORTHERN BLVD':'Northern Blvd',
'NORWOOD 205 ST':'Norwood - 205th St',
'NORWOOD AV':'Norwood Ave',
'NOSTRAND AV':'Nostrand Ave',
'OCEAN PKWY':'Ocean Pkwy',
'ORCHARD BEACH':'NaN',
'OZONE PK LEFFRT':'Ozone Park - Lefferts Blvd',
'PARK PLACE':'Park Pl',
'PARKCHESTER':'Parkchester',
'PARKSIDE AV':'Parkside Ave',
'PARSONS BLVD':'Parsons Blvd',
'PATH NEW WTC':'World Trade Center',
'PATH WTC':'World Trade Center',
'PATH WTC 2':'World Trade Center',
'PAVONIA/NEWPORT':'NaN',
'PELHAM BAY PARK':'Pelham Bay Park',
'PELHAM PKWY':'Pelham Pkwy',
'PENNSYLVANIA AV':'Pennsylvania Ave',
'PRESIDENT ST':'President St',
'PRINCE ST':'Prince St',
'PROSPECT AV':'Prospect Ave',
'PROSPECT PARK':'Prospect Park',
'QUEENS PLAZA':'Queens Plz',
'QUEENSBORO PLZ':'Queensboro Plz',
'RALPH AV':'Ralph Ave',
'RECTOR ST':'Rector St',
'RIT-MANHATTAN':'NaN',
'RIT-ROOSEVELT':'NaN',
'ROCKAWAY AV':'Rockaway Ave',
'ROCKAWAY BLVD':'Rockaway Blvd',
'ROCKAWAY PARK B':'Rockaway Park - Beach 116 St',
'ROOSEVELT ISLND':'Roosevelt Island - Main St',
'SARATOGA AV':'Saratoga Ave',
'SENECA AVE':'Seneca Ave',
'SHEEPSHEAD BAY':'Sheepshead Bay',
'SHEPHERD AV':'Shepherd Ave',
'SIMPSON ST':'Simpson St',
'SMITH-9 ST':'Smith - 9th Sts',
'SOUTH FERRY':'South Ferry',
'SPRING ST':'Spring St',
'ST LAWRENCE AV':'St Lawrence Ave',
'ST. GEORGE':'NaN',
'STEINWAY ST':'Steinway St',
'STERLING ST':'Sterling St',
'SUTPHIN BLVD':'Sutphin Blvd',
'SUTPHIN-ARCHER':'Sutphin Blvd - Archer Av',
'SUTTER AV':'Sutter Ave',
'SUTTER AV-RUTLD':'Sutter Ave - Rutland Road',
'THIRTY ST':'NaN',
'THIRTY THIRD ST':'NaN',
'TIMES SQ-42 ST':'Times Sq - 42nd St',
'TREMONT AV':'Tremont Ave',
'TWENTY THIRD ST':'NaN',
'UNION ST':'Union St',
'UTICA AV':'Utica Ave',
'V.CORTLANDT PK':'Van Cortlandt Park - 242nd St',
'VAN SICLEN AV':'Van Siclen Ave',
'VAN SICLEN AVE':'NaN',
'VERNON-JACKSON':'Vernon Blvd - Jackson Ave',
'W 4 ST-WASH SQ':'W 4th St - Washington Sq (Lower)',
'W 8 ST-AQUARIUM':'W 8th St - NY Aquarium',
'WAKEFIELD/241':'Wakefield - 241st St',
'WALL ST':'Wall St',
'WEST FARMS SQ':'West Farms Sq - E Tremont Av',
'WESTCHESTER SQ':'Westchester Sq - E Tremont Ave',
'WHITEHALL S-FRY':'Whitehall St',
'WHITLOCK AV':'Whitlock Ave',
'WILSON AV':'Wilson Ave',
'WINTHROP ST':'Winthrop St',
'WOODHAVEN BLVD':'Woodhaven Blvd',
'WOODLAWN':'Woodlawn',
'WORLD TRADE CTR':'World Trade Center',
'YORK ST':'York St',
'ZEREGA AV':'Zerega Ave',

}


In [122]:
turnstile_agg['station_map'] = turnstile_agg['Station'].map(dict_map)

In [123]:
turnstile_agg.head()

,Station,Date,Entries,Exits,entries_diff,exit_diff,station_map
1,1 AV,2016-01-03,1407817257,1469389492,14437,15092,1st Ave
2,1 AV,2016-01-04,1407839057,1469414011,21800,24519,1st Ave
3,1 AV,2016-01-05,1407861698,1469438036,22641,24025,1st Ave
4,1 AV,2016-01-06,1407885485,1469462526,23787,24490,1st Ave
5,1 AV,2016-01-07,1407909324,1469487185,23839,24659,1st Ave


In [124]:
#dataframe version
turnstile_df = pd.merge(left=turnstile_agg,right=station_locs,left_on='station_map',right_on='name')

In [125]:
#shapefile version
turnstile_gpdf = pd.merge(left=station_locs,right=turnstile_agg,left_on='name',right_on='station_map')

In [126]:
turnstile_df.head()

,Station,Date,Entries,Exits,entries_diff,exit_diff,station_map,line,name,notes,objectid,url,geometry
0,1 AV,2016-01-03,1407817257,1469389492,14437,15092,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
1,1 AV,2016-01-04,1407839057,1469414011,21800,24519,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
2,1 AV,2016-01-05,1407861698,1469438036,22641,24025,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
3,1 AV,2016-01-06,1407885485,1469462526,23787,24490,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
4,1 AV,2016-01-07,1407909324,1469487185,23839,24659,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)


In [127]:
turnstile_df.to_csv('turnstile2016_agg.csv')

In [130]:
turnstile_df.head()

,Station,Date,Entries,Exits,entries_diff,exit_diff,station_map,line,name,notes,objectid,url,geometry
0,1 AV,2016-01-03,1407817257,1469389492,14437,15092,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
1,1 AV,2016-01-04,1407839057,1469414011,21800,24519,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
2,1 AV,2016-01-05,1407861698,1469438036,22641,24025,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
3,1 AV,2016-01-06,1407885485,1469462526,23787,24490,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
4,1 AV,2016-01-07,1407909324,1469487185,23839,24659,1st Ave,L,1st Ave,L-all times,146.0,http://web.mta.info/nyct/service/,POINT (-73.98168087489128 40.73097497580066)
